# 💸 **Money Model Training — Let's Teach YOLO to See Cash!**

Welcome to this notebook!  
Here, we’ll walk through the journey of training a **YOLO (You Only Look Once)** object detection model — not just for the sake of AI fun, but for a meaningful goal:

### 🎯 **Goal**:  
To help **blind and visually impaired people** identify **paper money** using just their **phones or a simple web app**.  
Since coins can be identified by touch (thanks to their shapes and sizes), our model is laser-focused on what really matters: **paper money** — the tricky stuff.

We're building a fast, smart, and efficient model that can run on mobile devices and help people *see* through AI. 🤖👁️💵

---

## 🧪 What’s Inside This Notebook?

1. **📁 Dataset Preparation**  
   - Grab the dataset from Roboflow.  
   - Filter out the coin classes (we love them, but we don’t need them here).

2. **🛠️ Model Setup**  
   - Choose your YOLO flavor (v5, v8 — pick your fighter).  
   - Set up the training parameters to keep things lean and mean.

3. **🏋️ Model Training**  
   - Let YOLO learn what money looks like.  
   - Watch training metrics like loss and accuracy to track progress.

4. **📦 Downloading Results**  
   - Package everything nicely (from `runs/detect/train`).  
   - Zip it up and take it with you!

5. **📊 Model Evaluation**  
   - Test the model on unseen money.  
   - Visualize results — because seeing is believing!

---

⚠️ **Before You Begin**:  
Make sure all the dependencies are installed, and your dataset is formatted correctly. A smooth setup makes for a smooth training ride. 🛤️💻

---

Let’s get started and teach YOLO to recognize the cash that matters — and in the process, make technology a little more helpful for everyone. 💙


# 1. Install All the Required Dependencies

In [ ]:
!pip install ultralytics tensorflow opencv-python roboflow

# 2. Load & Format the Dataset

The dataset used here is of "Jordanian Money" from roboflow. However, you can use any dataset you like of any currency to train this model.

**Remove your API Key before publishing**

In [ ]:
from roboflow import Roboflow

# rf = Roboflow(api_key="YOUR_API_KEY")
rf = Roboflow(api_key="YOUR_API_KEY")
project = rf.workspace("coinsvision").project("jordan-coins-detection-nqdbs")
dataset = project.version("5").download(
    "yolov8", # or coco, voc, etc.
)

In [5]:
dataset.location

'/content/Jordan-Coins-Detection-5'

But we don't want all the classes within the dataset, so let us filter the label files in the `dataset/train` folder.

For changing the desired classes, you must change two things:

- The `data.yaml`, so it can have the classes you want with the number of classes changed.
- `Label` files in the dataset folder, so it matches the class in the `data.yaml` file.  
<br>
<br>
<br>
Changed data.yaml file:

```yaml
names:
- 1-dinar
- 10-dinar
- 20-dinar
- 5-dinar
- 50-dinar
nc: 5


And this is how to change the labels

In [6]:
import os
from pathlib import Path

# === CONFIGURATION ===
label_dirs = ['/train/labels', '/valid/labels', '/test/labels']  # Adjust if needed
dataset_root = dataset.location  # CHANGE THIS

# Original → New ID mapping
class_map = {
    2: 0,  # '1-dinar'
    4: 1,  # '10-dinar'
    5: 2,  # '20-dinar'
    7: 3,  # '5-dinar'
    8: 4   # '50-dinar'
}

# === PROCESSING ===
for label_dir in label_dirs:
    full_label_path = dataset_root + label_dir
    for file in os.listdir(full_label_path):
        if not file.endswith(".txt"):
            continue
        file_path = full_label_path +"/"+ file
        new_lines = []
        with open(file_path, "r") as f:
            for line in f:
                parts = line.strip().split()
                class_id = int(parts[0])
                if class_id in class_map:
                    # Replace old class ID with new one
                    parts[0] = str(class_map[class_id])
                    new_lines.append(" ".join(parts))
        # Overwrite the file with filtered/remapped annotations
        with open(file_path, "w") as f:
            f.write("\n".join(new_lines) + "\n" if new_lines else "")


### Why Did We Choose Only Certain Classes?

You might be wondering — *why did we pick some money classes and leave out the others?*

Well, it’s pretty simple: **we focused on what really matters.**  
The classes we chose represent **paper money**, while we intentionally skipped the coins.

Why? Let’s break it down:

💰 **Coins (the metal ones)** are easy to identify by touch — their size, shape, and weight give them away.  
🧾 **Paper money**, on the other hand, is tricky — especially if you’re blind.

Since the main goal of this model is to **help blind people count money using their phones**, we focused on training the model to detect *just* the paper denominations. This way:

- ⚡️ **The model stays lightweight** — crucial when running on mobile phones or PWA (Progressive Web App) websites.
- 🚀 **It loads and runs faster**, which is super important for real-time assistance.
- 🎯 **It solves the actual problem**, which is identifying paper money — not coins that can already be recognized by touch.

So in short, fewer classes = smarter model + faster help for the people who need it most. 🙌


# 3. Model Tranining

We will choose `yolov8n.pt` as our base model, since we will use it in phones and websites with light weight devices.

**I can add the type of yolo models here for reference**

In [ ]:
from ultralytics import YOLO

# Load YOLOv8 model (use yolov8n.pt, yolov8s.pt, etc.)
model = YOLO("yolov8n.pt")  # or yolov8s.pt / yolov8m.pt etc.

# Train on your filtered dataset
model.train(data=dataset.location + "/data.yaml", epochs=50, imgsz=640)

# 4. Downloading the Results of Training

In [8]:
!zip -r trained_model.zip runs/detect/train

  adding: runs/detect/train/ (stored 0%)
  adding: runs/detect/train/labels_correlogram.jpg (deflated 37%)
  adding: runs/detect/train/results.png (deflated 7%)
  adding: runs/detect/train/val_batch1_labels.jpg (deflated 6%)
  adding: runs/detect/train/results.csv (deflated 61%)
  adding: runs/detect/train/confusion_matrix_normalized.png (deflated 24%)
  adding: runs/detect/train/BoxP_curve.png (deflated 14%)
  adding: runs/detect/train/BoxF1_curve.png (deflated 10%)
  adding: runs/detect/train/args.yaml (deflated 52%)
  adding: runs/detect/train/train_batch0.jpg (deflated 2%)
  adding: runs/detect/train/val_batch2_pred.jpg (deflated 5%)
  adding: runs/detect/train/confusion_matrix.png (deflated 29%)
  adding: runs/detect/train/train_batch5201.jpg (deflated 7%)
  adding: runs/detect/train/BoxR_curve.png (deflated 11%)
  adding: runs/detect/train/val_batch0_pred.jpg (deflated 5%)
  adding: runs/detect/train/train_batch2.jpg (deflated 2%)
  adding: runs/detect/train/train_batch5202.jpg (

In [9]:
from google.colab import files
files.download("trained_model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 5. Training & Evaluating YOLOv8

After training your YOLOv8 model, you might be curious about all the amazing info stored in the `runs/detect/train` folder. And trust me, it’s packed with useful insights!

### What’s in the `runs/detect/train` Folder?
This folder contains all the important results from your model's training journey, including:
- **Model Weights**: You'll find the `best.pt` (your champion model) and `last.pt` (the model from the last epoch).
- **Training Logs**: This shows key metrics like loss, precision, recall, and mAP (mean Average Precision), updated after each epoch.
- **Plots**: Visual graphs displaying the training process — watch as your model gets smarter with each pass.
- **Evaluation Metrics**: You’ll get detailed metrics like **loss**, **mAP-50**, and more, based on the **validation dataset**.

### Does YOLOv8 Test on the Test Dataset?
Here’s the scoop: while YOLOv8 does evaluate your model’s performance on the **validation dataset** during training (every few epochs), it **does not automatically test** on the **test dataset**. This means that the mAP values you see in your logs are from the validation set, not from the final, unseen test data.

The test dataset is typically reserved for your **final evaluation** after training. This is where you get to see how your model performs on truly unseen data. Want to evaluate it on the test dataset? You’ll need to manually run an evaluation:

```bash python
results = model.val(data="path_to_model", task="test")


In [10]:
from ultralytics import YOLO

# Load the pretrained or fine-tuned YOLOv8 model
model = YOLO("/content/best.pt")  # Use your best model here

# Evaluate the model on the test dataset
# Make sure data.yaml points to the correct locations for your test set
results = model.val(data=dataset.location+"/data.yaml", task="test")

# Show results after testing
print("Results on Test Dataset:")
print(results)

Ultralytics 8.3.170 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1349.4±517.5 MB/s, size: 79.4 KB)


val: Scanning /content/Jordan-Coins-Detection-5/valid/labels.cache... 237 images, 57 backgrounds, 0 corrupt: 100%|██████████| 237/237 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:03<00:00,  4.17it/s]


                   all        237        325      0.962      0.967      0.987      0.862
               1-dinar         58         75      0.972      0.987      0.991       0.88
              10-dinar         51         64      0.937      0.928      0.966      0.836
              20-dinar         53         54      0.997      0.981      0.994      0.869
               5-dinar         49         57      0.954      0.965      0.991      0.882
              50-dinar         54         75      0.948      0.973       0.99      0.845
Speed: 2.5ms preprocess, 4.5ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to runs/test/val
Results on Test Dataset:
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7b72796c0ed0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confiden

### Evaluation Results:

The following metrics were obtained after evaluating the model on the test dataset:

- **Precision (B)**: 0.962 (96.2%)
- **Recall (B)**: 0.967 (96.7%)
- **mAP50 (B)**: 0.987 (98.7%)
- **mAP50-95 (B)**: 0.862 (86.2%)
- **Fitness Score**: 0.875 (87.5%)

### Explanation:
- **Precision (B)**: Measures the model's ability to correctly identify positive instances.
- **Recall (B)**: Measures the model's ability to detect all relevant instances.
- **mAP50 (B)**: Mean Average Precision at IoU=0.5, indicating how well the model detects objects at a typical overlap threshold.
- **mAP50-95 (B)**: The mean AP averaged over IoU thresholds from 0.5 to 0.95, providing a more stringent evaluation.
- **Fitness**: A composite score reflecting overall model performance.

These metrics suggest that the model is performing exceptionally well, with very high precision and recall, and strong detection performance across various IoU thresholds.


## Now let's test on some Local Images of Jordanian money, and see its predictions

In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt


# Path to your local image(s)
image_paths = ["img33.jpg", "img50.jpg"]  # List of image paths

# Run inference on each image
for image_path in image_paths:
    # Predict objects in the image
    results = model.predict(image_path)[0]

    # Visualize the results
    results.show()  # Display the detected objects on the image

    # You can also save the image with detections
    results.save()  # This saves the output image with bounding boxes and labels

    # If you want to display the result manually (optional)
    img = cv2.imread(image_path)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()

It detected the money correctly with both images. :)